In [162]:
import pandas as pd
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [163]:
#install tensorflow 2.8
!pip install tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [164]:
#ambil link project dataset
url = 'https://raw.githubusercontent.com/RashidMaulana/Anticede/main/Machine%20Learning/data/datatest.csv'
dataset = pd.read_csv(url)
dataset.head()

,original_text,pornografi,sara,radikalisme,pencemaran_nama_baik
0,1 BUKAN CuMa SPANDUK PROF VIDEO2 ORASI MEREKA ...,0.0,0.0,1.0,1.0
1,gua sange gatel yah tetek memek Y d...,1.0,0.0,0.0,0.0
2,pertama kali denger lagunya enak bangitu in d...,0.0,0.0,0.0,0.0
3,astajim ini pasti yang kasih penghargaan ke i...,0.0,0.0,0.0,1.0
4,kalo keberagamannya ada tapi ga dianggep ban...,0.0,0.0,0.0,0.0


In [165]:
dataset.tail()

,original_text,pornografi,sara,radikalisme,pencemaran_nama_baik
4995,Puasa ora sok sokan gaweni story bukber kantor...,1.0,0.0,0.0,1.0
4996,Kalau sudah begini rame - rame saling cuci tan...,0.0,0.0,1.0,1.0
4997,4 Kebijakan Presiden Jokowi yang Bikin PNS Bah...,0.0,0.0,0.0,0.0
4998,seorang filsuf pun harus feel safe ...,0.0,0.0,0.0,0.0
4999,museum date tapi museum pengkhianatan PKI (kom...,0.0,0.0,0.0,0.0


In [166]:
# Change text to lowercase
def proses_data(text):
    newText = text.lower()
    return newText
#ini case kalau semua mau ngerun semua data Komputerku kentang maka ngefreeze)
# dataset['comment_text'].apply(lambda x: print(x))
#ini case kalau semua mau ngerun 5 data pertama
testData = dataset['original_text']
testData = testData.apply(lambda x: proses_data(x))
testData.head()


0    1 bukan cuma spanduk prof video2 orasi mereka ...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [167]:
#menghilangkan link dan tag html
import re
def link_remover(text):
    url_regex = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    return url_regex.sub('', text)
testData = testData.apply(lambda x: link_remover(x))
testData.head()

0    1 bukan cuma spanduk prof video2 orasi mereka ...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [168]:
#menghilangkan kalimat bersama tag
import re
def tag_remover(text):
    tag_regex = re.compile(r'<[^>]+>')
    return tag_regex.sub('', text)
testData = testData.apply(lambda x: tag_remover(x))
testData.head()

0    1 bukan cuma spanduk prof video2 orasi mereka ...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [169]:
# menghilangkan tanda baca dan karakter spesial
import re
def karakter_spesial(text):
    newText = re.sub('[^a-zA-Z0-9\s]', '', text)
    return newText
testData = testData.apply(lambda x: karakter_spesial(x))
testData.head()

0    1 bukan cuma spanduk prof video2 orasi mereka ...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [170]:
# menghilangkan spasi berlebihan
def Spasi_berlebihan(text):  
    newText = re.sub('\s', ' ', text)
    return newText
testData = testData.apply(lambda x: Spasi_berlebihan(x))
testData.head()

0    1 bukan cuma spanduk prof video2 orasi mereka ...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [171]:
def angka_berlebihan(text):
    newText = re.sub('[0-9]', '', text)
    return newText
testData = testData.apply(lambda x: angka_berlebihan(x))
testData.head()

0     bukan cuma spanduk prof video orasi mereka  b...
1    gua sange       gatel yah tetek      memek y d...
2    pertama  kali denger lagunya enak bangitu in d...
3    astajim  ini pasti yang kasih penghargaan ke i...
4     kalo keberagamannya ada tapi ga dianggep  ban...
Name: original_text, dtype: object

In [172]:
dataset['pornografi'].value_counts()

0.0    4555
1.0     438
Name: pornografi, dtype: int64

In [173]:
dataset['radikalisme'].value_counts()

0.0    4406
1.0     587
Name: radikalisme, dtype: int64

In [174]:
dataset['pencemaran_nama_baik'].value_counts()

0.0    2694
1.0    2299
Name: pencemaran_nama_baik, dtype: int64

In [175]:
dataset['sara'].value_counts()

0.0    4479
1.0     515
Name: sara, dtype: int64

In [176]:
#melatih data train dengan tensorflow
url_train_data = 'https://raw.githubusercontent.com/RashidMaulana/Anticede/main/Machine%20Learning/data/datatrain.csv'
train_data = pd.read_csv(url_train_data)

# Change text to lowercase
def proses_data(text):
    newText = text.lower()
    return newText
traindata = train_data['original_text']
traindata = traindata.apply(lambda x: proses_data(x))

#menghilangkan link dan tag html
def link_remover(text):
    url_regex = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
    return url_regex.sub('', text)
traindata = traindata.apply(lambda x: link_remover(x))

#menghilangkan kalimat bersama @name
def name_remover(text):
    name_regex = re.compile(r'@[\w_-]+')
    return name_regex.sub(' ', text)
traindata = traindata.apply(lambda x: name_remover(x))

#menghilangkan tanda baca dan karakter spesial
def karakter_spesial(text):
    newText = re.sub('[^a-zA-Z0-9\s]', ' ', text)
    return newText
traindata = traindata.apply(lambda x: karakter_spesial(x))

#menghilang angka yang nempel dengan kalimat
def angka_berlebihan(text):
    newText = re.sub('[0-9]', '', text)
    return newText
traindata = traindata.apply(lambda x: angka_berlebihan(x))

#menghilangkan spasi berlebihan
def Spasi_berlebihan(text):  
    newText = re.sub('\s+', '', text)
    newText = re.sub('^\s+', '', text)
    return newText
traindata = traindata.apply(lambda x: Spasi_berlebihan(x))
traindata.tail()

995    nah loh  masih menyebut jokowi antek asing   c...
996    ade armando dipolisikan soal postingan     aza...
997    fifa siasat isu  suara monyet     aksi peranci...
998                           ya angkot lebih goblog    
999    njirr pahanya kok berlemak nabati gitu sih mas...
Name: original_text, dtype: object

In [177]:
#oov tokenizer
tokenizer = Tokenizer(num_words = 15000, oov_token="<OOV>")
tokenizer.fit_on_texts(testData)
word_index = tokenizer.word_index #menampilkan tabel kata yang ada di dataset
print(f'number of words in word_index: {len(word_index)}')
# Print the word index
print(f'word_index: {word_index}')
print()
# Generate and pad the sequences
sequences = tokenizer.texts_to_sequences(testData)
padded = pad_sequences(sequences, padding='post')
# Print a sample headline
index = 2
print(f'sample headline: {testData[index]}')
print(f'padded sequence: {padded[index]}')
print()

number of words in word_index: 15773
word_index: {'<OOV>': 1, 'yang': 2, 'dan': 3, 'di': 4, 'itu': 5, 'ini': 6, 'ada': 7, 'orang': 8, 'dari': 9, 'dengan': 10, 'ya': 11, 'jadi': 12, 'presiden': 13, 'tapi': 14, 'juga': 15, 'gak': 16, 'ga': 17, 'bisa': 18, 'aja': 19, 'untuk': 20, 'tidak': 21, 'saya': 22, 'sama': 23, 'ke': 24, 'jokowi': 25, 'gue': 26, 'agama': 27, 'kalo': 28, 'mau': 29, 'nya': 30, 'lagi': 31, 'lu': 32, 'dia': 33, 'aku': 34, 'apa': 35, 'kita': 36, 'indonesia': 37, 'islam': 38, 'karena': 39, 'mereka': 40, 'kalau': 41, 'jangan': 42, 'sudah': 43, 'bukan': 44, 'udah': 45, 'dalam': 46, 'akan': 47, 'lain': 48, 'lo': 49, 'lebih': 50, 'pada': 51, 'kafir': 52, 'semua': 53, 'banyak': 54, 'pak': 55, 'asing': 56, 'masih': 57, 'kan': 58, 'kok': 59, 'cina': 60, 'si': 61, 'adalah': 62, 'atau': 63, 'cebong': 64, 'tak': 65, 'buat': 66, 'punya': 67, 'cuma': 68, 'negara': 69, 'sekarang': 70, 'gitu': 71, 'lah': 72, 'baru': 73, 'seperti': 74, 'ganti': 75, 'anak': 76, 'terus': 77, 'rakyat': 78, 

In [178]:
# menampilkannya dalam bentuk integer
sequences = tokenizer.texts_to_sequences(testData)
print(sequences)

[[44, 68, 1476, 755, 218, 4522, 40, 1243, 40, 3, 48, 48, 5, 45, 159, 224, 22, 783, 23, 78, 2, 57, 203, 45, 224, 7, 1344, 9, 784, 2, 307, 1244, 264, 75, 268, 57, 1613, 16, 7, 72, 29, 1244, 264, 19, 473, 2367, 264], [86, 454, 4523, 424, 785, 90, 1614, 4, 2368, 265, 720, 4, 992, 385, 2049, 993, 1477, 98, 113, 2050, 332, 98, 113, 880, 17, 424, 1615, 881], [386, 152, 1080, 1616, 269, 6828, 443, 3, 219, 6829, 6830, 197, 153, 2811, 86, 1616, 39, 269, 4, 1080, 160, 153, 51, 2811, 1616, 1617, 1166, 443], [2369, 6, 128, 2, 212, 4524, 24, 370, 2370, 62, 161, 56, 6831, 3, 179, 882, 4525, 6832], [28, 6833, 7, 14, 17, 6834, 54, 14, 6835, 7, 14, 6836, 213, 132, 19, 160, 3469, 31, 937], [7, 4526, 4527, 994, 85, 28, 33, 4526, 19, 45, 541], [2812, 76, 6, 1478, 786, 406, 150, 193, 154, 500, 6837, 6838, 4528, 80, 2371, 721, 2051], [6839, 444, 4529, 2052, 938, 121, 637, 39, 4530, 2053, 12, 2813, 144, 33, 45, 1479, 98], [177, 4531, 679, 787, 939, 357, 2814, 610, 2, 445, 722, 2372, 3, 2372, 296, 2054, 145, 8

In [179]:
from tensorflow.keras.preprocessing.text import Tokenizer
#initialize the tokenizer class
tokenizer = Tokenizer()
tokenizer.fit_on_texts(traindata)
word_index = tokenizer.word_index
print(f'number of words in word_index: {len(word_index)}')
print(f'word_index: {word_index}')

number of words in word_index: 5593
word_index: {'yang': 1, 'di': 2, 'dan': 3, 'itu': 4, 'ada': 5, 'ini': 6, 'orang': 7, 'dengan': 8, 'gak': 9, 'aja': 10, 'ya': 11, 'tapi': 12, 'ga': 13, 'tidak': 14, 'dari': 15, 'presiden': 16, 'jadi': 17, 'mau': 18, 'gue': 19, 'sama': 20, 'bisa': 21, 'ke': 22, 'juga': 23, 'aku': 24, 'kalo': 25, 'saya': 26, 'jokowi': 27, 'islam': 28, 'untuk': 29, 'agama': 30, 'kita': 31, 'dia': 32, 'lagi': 33, 'apa': 34, 'nya': 35, 'jangan': 36, 'indonesia': 37, 'lu': 38, 'akan': 39, 'sudah': 40, 'udah': 41, 'mereka': 42, 'kalau': 43, 'karena': 44, 'bukan': 45, 'dalam': 46, 'pak': 47, 'lo': 48, 'cebong': 49, 'si': 50, 'gua': 51, 'semua': 52, 'negara': 53, 'cina': 54, 'asing': 55, 'banyak': 56, 'kafir': 57, 'lebih': 58, 'lain': 59, 'pada': 60, 'kami': 61, 'kok': 62, 'baru': 63, 'tak': 64, 'kamu': 65, 'buat': 66, 'atau': 67, 'cuma': 68, 'kan': 69, 'hanya': 70, 'harus': 71, 'sih': 72, 'masih': 73, 'saja': 74, 'kristen': 75, 'kayak': 76, 'tuh': 77, 'muslim': 78, 'rakyat': 

In [180]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#initialize the sequences list
sequences = tokenizer.texts_to_sequences(traindata)
padded = pad_sequences(sequences, padding='post')
print(f'shape of padded sequences: {padded.shape}')
label = train_data['pornografi']
label.head()
#print the sample headline
index = 2
print(f'sample headline: {traindata[index]}')
inputs = padded[:-1]
labels = padded[1:]
# Print the shape of inputs and labels
print(f'shape of inputs: {inputs.shape}')

shape of padded sequences: (1000, 357)
sample headline: iya kesel udah kalah debat gua diblok lagi      kelihatan yang tolol siapa  kesel gedekk gua 
shape of inputs: (999, 357)


In [181]:
#data preprocessing
vocab_size = 1000
max_length = 100
padding_type = 'post'
oov_token = '<OOV>'
# Initialize tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(traindata)
word_index = tokenizer.word_index
print(f'number of words in word_index: {len(word_index)}')
# Print the word index
print(f'word_index: {word_index}')
print()
# Generate and pad the testdata sequences
sequences = tokenizer.texts_to_sequences(testData)
padded = pad_sequences(sequences, padding=padding_type, maxlen=max_length)
# Print a sample headline
index = 4
print(f'sample headline: {testData[index]}')
print(f'padded sequence: {padded[index]}')
print()


number of words in word_index: 5594
word_index: {'<OOV>': 1, 'yang': 2, 'di': 3, 'dan': 4, 'itu': 5, 'ada': 6, 'ini': 7, 'orang': 8, 'dengan': 9, 'gak': 10, 'aja': 11, 'ya': 12, 'tapi': 13, 'ga': 14, 'tidak': 15, 'dari': 16, 'presiden': 17, 'jadi': 18, 'mau': 19, 'gue': 20, 'sama': 21, 'bisa': 22, 'ke': 23, 'juga': 24, 'aku': 25, 'kalo': 26, 'saya': 27, 'jokowi': 28, 'islam': 29, 'untuk': 30, 'agama': 31, 'kita': 32, 'dia': 33, 'lagi': 34, 'apa': 35, 'nya': 36, 'jangan': 37, 'indonesia': 38, 'lu': 39, 'akan': 40, 'sudah': 41, 'udah': 42, 'mereka': 43, 'kalau': 44, 'karena': 45, 'bukan': 46, 'dalam': 47, 'pak': 48, 'lo': 49, 'cebong': 50, 'si': 51, 'gua': 52, 'semua': 53, 'negara': 54, 'cina': 55, 'asing': 56, 'banyak': 57, 'kafir': 58, 'lebih': 59, 'lain': 60, 'pada': 61, 'kami': 62, 'kok': 63, 'baru': 64, 'tak': 65, 'kamu': 66, 'buat': 67, 'atau': 68, 'cuma': 69, 'kan': 70, 'hanya': 71, 'harus': 72, 'sih': 73, 'masih': 74, 'saja': 75, 'kristen': 76, 'kayak': 77, 'tuh': 78, 'muslim': 7

In [182]:
#generate and pad datatrain sequences
sequences_1 = tokenizer.texts_to_sequences(traindata)
padded_1 = pad_sequences(sequences, padding=padding_type, maxlen=max_length)
print(f'sample headline: {traindata[index]}')
print(f'padded sequence: {padded[index]}')
print()

sample headline: ga usah tunggu lama dadamu yang disumpel silikon bakal mbledug karena alloh ga suka umbar yang wajib ditutup 
padded sequence: [ 26   1   6  13  14   1  57  13   1   6  13   1 168 131  11 103   1  34
 752   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]



In [183]:
#convert to numpy array
padded = np.array(padded)
padded_1 = np.array(padded_1)
print(f'padded shape: {padded.shape}')
print(f'padded_1 shape: {padded_1.shape}')

#split data train to train and validation
from sklearn.model_selection import train_test_split
train, validation = train_test_split(padded_1, test_size=0.2, random_state=42)
print(f'train shape: {train.shape}')
print(f'validation shape: {validation.shape}')
print()

padded shape: (5000, 100)
padded_1 shape: (5000, 100)
train shape: (4000, 100)
validation shape: (1000, 100)



In [187]:
import tensorflow as tf

# Parameters
embedding_dim = 16
lstm_dim = 32
dense_dim = 24

# Model Definition with LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_lstm.summary()



Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 100, 16)           16000     
                                                                 
 bidirectional (Bidirectiona  (None, 64)               12544     
 l)                                                              
                                                                 
 dense_19 (Dense)            (None, 24)                1560      
                                                                 
 dense_20 (Dense)            (None, 1)                 25        
                                                                 
Total params: 30,129
Trainable params: 30,129
Non-trainable params: 0
_________________________________________________________________


In [191]:
NUM_EPOCHS = 10

# Train the model
history_lstm = model_lstm.fit(padded, padded_1, epochs=NUM_EPOCHS, validation_data=(sequences, sequences_1))


Epoch 1/10


ValueError: ignored